# CRUD Operations in students

Lo primero para evitar conflictos es darle otro nombre al momento de importar los datos de los estudiantes:

`from datos import STUDENTS as students_db`

Y continuaremos implementando CRUD(GET, POST, PUT, DELETE) en `students.py`, y de ser necesario crearemos nuevos endpoints.

## POST

La primera implementacion, usando la clase `Body` de FASTAPI, que se usa para:


- **Indicar explícitamente que un parámetro debe venir del cuerpo de la solicitud**. Esto es crucial cuando los parámetros son tipos simples (como int o str), ya que de otro modo FastAPI los interpretaría como parámetros de consulta.

- **Añadir validaciones y metadatos adicionales a los parámetros del cuerpo**, de forma similar a como ``Query`` y ``Path`` lo hacen para sus respectivos tipos de parámetros.

```py
@router.post("/student", status_code=201)
async def create_student(id:int = Body(gt=1), name:str = Body()):
    """
    Crea un nuevo estudiante en la BD, para verificar que no existe el nuevo estudiante,
    busca por su id, si hay un conflicto retorna 409
    """
    for student in students_db:
        if student['id'] == id:
            raise HTTPException(status_code=409, detail={"message": f"usuario {id} ya existe"})
        
    new_student = {'id':id, 'name':name}
    students_db.append(new_student)

    return JSONResponse(status_code=201, content=new_student)
```

Aqui solamente estamos añadiendo id y name.

### Usando BaseModel

Mientras que ``Body`` es excelente para parámetros individuales, en un escenario real para crear un estudiante, generalmente querrías enviar un objeto JSON completo que represente al estudiante. Para esto, FastAPI recomienda fuertemente usar modelos de *Pydantic* (``BaseModel``).


```py
class StudentCreate(BaseModel):
    id: int = Field(ge=1)
    name: str = Field(min_length=3)
    lastName: str = Field(min_length=3)
    governmentId: str = Field(min_length=1)
    typeOfDocument: str = Field("CC", max_length=2)
    DOB: Optional[str] = None
    location: str = Field("Bogota")
```

La clase ``Field`` en Pydantic (y por extensión, en FastAPI) es una función de utilidad que te permite añadir validaciones y metadatos adicionales a los atributos de tus modelos Pydantic.



```py
@router.post("/student", status_code=201)
async def create_student(new_student: StudentCreate):
    """
    Crea un nuevo estudiante en la BD, para verificar que no existe el nuevo estudiante,
    busca por su id, si hay un conflicto retorna 409
    """
    for student in students_db:
        if student['id'] == new_student.id:
            raise HTTPException(status_code=409, detail={"message": f"usuario {new_student.id} ya existe"})

    # Convierte la instancia de Pydantic a un diccionario
    new_student_dict = new_student.model_dump()

    students_db.append(new_student_dict)

    return JSONResponse(status_code=201, content=new_student_dict)
```



### PUT y PATCH

En el contexto de las APIs REST, PUT y PATCH son métodos HTTP para actualizar recursos, pero difieren en su enfoque. PUT se utiliza para reemplazar completamente un recurso con los datos proporcionados, mientras que PATCH se utiliza para aplicar modificaciones parciales a un recurso. 

Una operación PATCH está diseñada para realizar actualizaciones parciales. Esto significa que el cliente enviará solo los campos que desea modificar, no necesariamente todos los campos del recurso

#### PATCH

Se crea una nueva clase basada en *Basemodel*, por ahora solo con cuatro atributos. Es importante aclarar que *Optional*, dice que es opcional, sin embargo, hubo muchos bugs, y finalmente se decidio incluir `None`, de otra forma indicaba al momento de hacer la peticion que era requerido.

```py
class StudentUpdate(BaseModel):
    name: Optional[str] = Field(None, min_length=3)
    lastName: Optional[str] = Field(None, min_length=3)
    governmentId: Optional[str] = Field(None, min_length=1)
    typeOfDocument: Optional[str] = Field(None, max_length=3)
```

##### Primera Implementacion:

```py
@router.patch("/student/{student_id_to_patch}", status_code=200)
async def update_student(student_id_to_patch: int = Path(gt=0, description="ID del estudiante a actualizar"),
                        student_update: StudentUpdate = Body(description="campos a actualizar" ) ):
    

    for student in students_db:
        if student['id'] == student_id_to_patch:
            # Convierte la instancia de Pydantic a un diccionario
            new_student_dict = student_update.model_dump() 👈
#            new_student_dict = student_update.model_dump(exclude_unset=True, mode='json')
            print('hello: student update')
            print(student_update)

            for key in new_student_dict:
                student[key] = new_student_dict[key]
            
            return JSONResponse(status_code=200, content={"message": "Estudiante Actualizado Exitosamente", "student": student })
    
    message = f"estudiante con id {student_id_to_patch} ¡no existe!"
    raise HTTPException(status_code=404, detail={"message":message}) 
```
#### bug fixing:

El problema con esta implementacion es que si solo se envia en mi peticion, lastName y governmentId:

`curl -X 'PATCH' \
  'http://localhost:8000/student/19' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{
  "lastName": "baab",
  "governmentId": "string"
}'`

Al final, los campos que no se enviaron, los llenara con Null, o None:

![image](https://imgur.com/Hwquz2l.png)

![image](https://imgur.com/XJ7z5I4.png)

Por eso es importante cambiar esta linea:

`new_student_dict = student_update.model_dump()` a `new_student_dict = student_update.model_dump(exclude_unset=True, mode='json')`

``model_dump()`` sin ``exclude_unset=True``: Cuando llamas a ``student_update.model_dump()`` sin el argumento exclude_unset=True, Pydantic convierte TODOS los campos del modelo a un diccionario, incluyendo aquellos que el cliente no envió y que por lo tanto tienen un valor ``None`` (porque son Optional).

#### Documentacion

¿Por qué los ``BaseModel`` aparecen en ``/docs`` como "Schemas"?

Cuando defines clases que heredan de ``pydantic.BaseModel`` en tu aplicación FastAPI, le estás proporcionando a FastAPI una definición estructurada de los datos que esperas recibir o enviar. FastAPI, en su núcleo, utiliza esta información para generar automáticamente la documentación interactiva de tu API, que puedes ver en ``/docs`` (Swagger UI) o /redoc.